In [4]:
from __future__ import print_function

from miscc.config import cfg, cfg_from_file
from trainer import condGANTrainer as trainer
from easydict import EasyDict as edict
import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
import pickle

import torch
import torchvision.transforms as transforms
import torch.nn as nn

from pytorch_pretrained_bert import BertTokenizer

In [2]:
from datasets_bert import TextDataset, prepare_data

In [3]:
image_transform = transforms.Compose([
        transforms.Resize(int(299 * 76 / 64)),
        transforms.RandomCrop(299),
        transforms.RandomHorizontalFlip()])
dataset = TextDataset('../data/birds', 'train',
                          base_size=299,
                     transform=image_transform)

Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
BertTokenizer.from_pretrained('bert-base-uncased').ids_to_tokens[1040]

'd'

In [4]:
dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=8, drop_last=True,
        shuffle=True, num_workers=1)

In [5]:
data_iter = iter(dataloader)
data = next(data_iter)

In [31]:
imgs, captions, cap_lens, \
            class_ids, keys = prepare_data(data)

In [22]:
with open('../data/birds/captions_bert.pickle', 'rb') as f:
    x = pickle.load(f)
    train_captions, test_captions = x[0], x[1]
    ixtoword, wordtoix = x[2], x[3]

In [30]:
for i in BertTokenizer.from_pretrained('bert-base-uncased').ids_to_tokens:
    if BertTokenizer.from_pretrained('bert-base-uncased').ids_to_tokens[i] == '[CLS]':
        print(i)
        break

101


In [35]:
BertTokenizer.from_pretrained('bert-base-uncased').ids_to_tokens[103]

'[MASK]'